# Experiments for HaliVer

In [1]:
from preprocess import *
from typing import List, Optional, Dict, Tuple, Any, Union
import time
import datetime

In [2]:
n = 1 # repetitions
t = 2*60 # timeout
load_results = False # If this is true, we do not run experiments, but only load them from file
load_prefix = "results/original"
save_prefix = "results/" + datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')

In [ ]:
print(save_prefix)

In [13]:
def runCommand(command: str, verbose: bool = False, timeout: Optional[int] = None
               , logfile: Optional[str] = None)-> Tuple[float, Result]:
    start = time.time()
    if(logfile):
        with open(logfile, "a") as f:
            f.write("==============\n")
            f.write(command)
            f.write("\n==============\n")
    
    try:
        p = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, timeout=timeout)
    except subprocess.TimeoutExpired as to:
        print(f"Process was stopped after {to.timeout} seconds")
        if(to.stdout):
            print(to.stdout.decode('UTF-8'))
        end = time.time()
        if(logfile):
            with open(logfile, "a") as f:
                f.write(to.stderr.decode('UTF-8'))
                f.write(to.stdout.decode('UTF-8'))
        return end - start, Result.TimeOut
        
    end = time.time()
    if(logfile):
        with open(logfile, "a") as f:
            f.write(p.stderr.decode('UTF-8'))
            f.write(p.stdout.decode('UTF-8'))
    if(p.returncode != 0):
        print(p.stdout.decode('UTF-8'))
        print(f"Return code was {p.returncode}")
        return end - start, Result.Fail
    elif verbose:
        print(p.stdout.decode('UTF-8'))
        
    return end - start, Result.Pass

In [15]:
runCommand('vct build/blur_front_pvl.pvl' ,logfile='temp.log')

(6.552173137664795, <Result.Pass: 'pass'>)

In [ ]:
versions: Dict[str, List[str]] = {
    'blur' : ['0','1','2','3'],
    'hist' : ['0','1','2','3'],
    'conv_layer' : ['0','1','2','3'],
    'gemm' : ['0','1','2','3'],
}
experiments = Experiments(versions, repetitions = n, timeout = t)

In [ ]:
from typing import List, Optional, Dict, Tuple, Any, Union
from enum import Enum
import subprocess
import os
import time
import json

## Blur

In [ ]:
name = 'blur'

In [ ]:
if(not load_results):
    experiments.make_files(name)

In [ ]:
if(not load_results):
    experiments.front_end(name)

In [ ]:
if(not load_results):
    experiments.back_end(name)

In [ ]:
if(not load_results):
    experiments.save_results(save_prefix, name)

## Hist

In [ ]:
name = 'hist'

In [ ]:
if(not load_results):
    experiments.make_files(name)

In [ ]:
if(not load_results):
    experiments.front_end(name)

In [ ]:
if(not load_results):
    experiments.back_end(name)

In [ ]:
if(not load_results):
    experiments.save_results(save_prefix, name)

## Conv

In [ ]:
name = 'conv_layer'

In [ ]:
if(not load_results):
    experiments.make_files(name)

In [ ]:
if(not load_results):
    experiments.front_end(name)

In [ ]:
if(not load_results):
    experiments.back_end(name)

In [ ]:
if(not load_results):
    experiments.save_results(save_prefix, name)

## Gemm

In [ ]:
name = 'gemm'

In [ ]:
if(not load_results):
    experiments.make_files(name)

In [ ]:
if(not load_results):
    experiments.front_end(name)

In [ ]:
if(not load_results):
    experiments.back_end(name)

In [ ]:
if(not load_results):
    experiments.save_results(save_prefix, name)

# Result table

In [ ]:
if(load_results):
    experiments.load_all_results("original")

In [ ]:
directivesUsed: Dict[str, Dict[str,str]] = {'blur' : {"1" : "\{f,p\}", "2": "\{c,p,r,s\}", "3": "\{c,p,s,st\}"},
       'hist' : {"1" : "\{c,p,r,u\}", "2": "\{c,p,r,u\}", "3": "\{c,cl,p,r,s,u\}"},
       'conv_layer' : {"1" : "\{c,f,p,u\}", "2": "\{p,r,s,u\}", "3": "\{c,p,r,s,u\}"},
       'gemm' : {"1" :"\{c,p,r,s\}","2": "\{c,p,r,s,u\}", "3": "\{c,f,p,r,s,u\}" }}

In [ ]:
halideLoC: Dict[str, int] = {'blur': 5, 'hist': 29, 'conv_layer' : 9, 'gemm' : 18}
halideAnn: Dict[str, int] = {'blur': 2, 'hist': 6, 'conv_layer' : 7, 'gemm' : 6}
scheduleLoC: Dict[str, Dict[str,int]] = {'blur' : {"1" : 2, "2": 5, "3": 6},
                                         'hist' : {"1" : 4, "2": 6, "3": 16},
                                         'conv_layer' : {"1" : 4, "2": 6, "3": 15},
                                         'gemm' : {"1": 8, "2": 19, "3": 23 }}

In [ ]:
print(experiments.make_table(directivesUsed, halideLoC, halideAnn, scheduleLoC))

In [ ]:
experiments.save_table(directivesUsed, halideLoC, halideAnn, scheduleLoC)
!pdflatex table.tex
PDF('table.pdf',size=(950,500))

# End